In [7]:
import pandas as pd
def print_ratio(df,x_values,save_name):
    # 计算每个组合的平均return
    avg_return = df.groupby(['algorithm', 'train mode', 'test mode'])['ep ret'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_return['algorithm'].unique()

    for algo in algorithms:
        algo_data = avg_return[avg_return['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        for x in x_values:
            before_condition = (avg_return['train mode'] == x - 1) & (avg_return['test mode'] == x - 1)
            max_ret = avg_return[before_condition]['ep ret'].max()
            
            # condition after training on new task
            after_condition = (algo_data['train mode'] == x) & (algo_data['test mode'] == x - 1)
            after_resutls = algo_data[after_condition]
            
            # condition before training on new task
            before_condition = (algo_data['train mode'] == x - 1) & (algo_data['test mode'] == x - 1)
            before_results = algo_data[before_condition]
            
            if not after_resutls.empty and not before_results.empty:
                performance_before = before_results['ep ret'].values[0] / max_ret
                performance_after = after_resutls['ep ret'].values[0] / max_ret
                # ratio = (performance_before - performance_after)
                # ratio = (performance_before - performance_after) / performance_before
                ratio = performance_before - performance_after
                algo_ratios[f'forgetting on {x}'] = ratio 
            else:
                algo_ratios[f'forgetting on {x}'] = None  
        
        # algo_ratios_df = pd.DataFrame(algo_ratios)
        # result_df = pd.concat([result_df, algo_ratios_df], ignore_index=True)
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = numeric_df.mean(axis=1)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)
    
def print_final_performance(df,x_values,save_name):
    # 计算每个组合的平均return
    avg_return = df.groupby(['algorithm', 'train mode', 'test mode'])['ep ret'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_return['algorithm'].unique()
    train_mode = x_values[-1]
    for algo in algorithms:
        algo_data = avg_return[avg_return['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        # 遍历每个x值
        for x in x_values:
            cur_condition = (avg_return['train mode'] == x) & (avg_return['test mode'] == x)
            max_ret = avg_return[cur_condition]['ep ret'].max()
            
            # 筛选出train mode = x, test mode = x-1的行
            condition1 = (algo_data['train mode'] == train_mode) & (algo_data['test mode'] == x)
            result1 = algo_data[condition1]
            
            # 筛选出train mode = x-1, test mode = x-1的行
            prev_condition = (algo_data['train mode'] == x) & (algo_data['test mode'] == x)
            result2 = algo_data[prev_condition]
            
            # 如果两组数据都存在，则计算比值
            if not result1.empty and not result2.empty:
                ratio = result1['ep ret'].values[0] / max_ret
                algo_ratios[f'{x}'] = ratio  # 动态列名
            else:
                algo_ratios[f'{x}'] = 0  # 如果没有匹配的数据，则设为None
        
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = numeric_df.mean(axis=1)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)
import pandas as pd
def print_ratio_MetaWorld(df,x_values,save_name):
    # 计算每个组合的平均return
    avg_success = df.groupby(['algorithm', 'train task', 'test task'])['success'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_success['algorithm'].unique()
    train_mode = x_values[-1]
    for algo in algorithms:
        algo_data = avg_success[avg_success['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        # 遍历每个x值
        for x in x_values:
            # cur_condition = (avg_success['train task'] == x) & (avg_success['test task'] == x)
            
            # 筛选出train mode = x, test mode = x-1的行
            condition1 = (algo_data['train task'] == train_mode) & (algo_data['test task'] == x)
            after_result = algo_data[condition1]
            
            # 筛选出train mode = x-1, test mode = x-1的行
            prev_condition = (algo_data['train task'] == x) & (algo_data['test task'] == x)
            before_result = algo_data[prev_condition]
            
            # 如果两组数据都存在，则计算比值
            if not after_result.empty and not before_result.empty:
                if before_result['success'].values[0] == 0:
                    algo_ratios[f'{x}'] = before_result['success'].values[0] - after_result['success'].values[0]  # 动态列名
                else:
                    algo_ratios[f'{x}'] = (before_result['success'].values[0] - after_result['success'].values[0]) / before_result['success'].values[0]  # 动态列名
            else:
                algo_ratios[f'{x}'] = 0  # 如果没有匹配的数据，则设为None
        
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = numeric_df.mean(axis=1)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)
    

def print_final_performance_MetaWorld(df,x_values,save_name):
    # 计算每个组合的平均return
    avg_success = df.groupby(['algorithm', 'train task', 'test task'])['success'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_success['algorithm'].unique()
    train_mode = x_values[-1]
    for algo in algorithms:
        algo_data = avg_success[avg_success['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        # 遍历每个x值
        for x in x_values:
            cur_condition = (avg_success['train task'] == x) & (avg_success['test task'] == x)
            
            # 筛选出train mode = x, test mode = x-1的行
            condition1 = (algo_data['train task'] == train_mode) & (algo_data['test task'] == x)
            result1 = algo_data[condition1]
            
            # 筛选出train mode = x-1, test mode = x-1的行
            prev_condition = (algo_data['train task'] == x) & (algo_data['test task'] == x)
            result2 = algo_data[prev_condition]
            
            # 如果两组数据都存在，则计算比值
            if not result1.empty and not result2.empty:
                algo_ratios[f'{x}'] = result1['success'].values[0]  # 动态列名
            else:
                algo_ratios[f'{x}'] = 0  # 如果没有匹配的数据，则设为None
        
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = numeric_df.mean(axis=1)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)

In [ ]:
import pandas as pd

# 读取CSV文件
# df = pd.read_csv('eval_Freeway/eval_results.csv')
# for x in range(1,9):
#     x_values = range(x)
#     print_final_performance(df, x_values, f'eval_Freeway/{x}_final_perf_results')

# df = pd.read_csv('eval_SpaceInvaders/eval_results.csv')
# for x in range(1,11):
#     x_values = range(x)
#     print_final_performance(df, x_values, f'eval_SpaceInvaders/{x}_final_perf_results')

df = pd.read_csv('eval_MetaWorld/eval_results.csv')
print(df.keys())
for x in range(1,21):
    x_values = range(x)
    print_final_performance_MetaWorld(df, x_values, f'eval_MetaWorld/{x}_final_perf_results')

In [8]:
import pandas as pd

# 读取CSV文件
# df = pd.read_csv('eval_Freeway/eval_results.csv')
# for x in range(1,9):
#     x_values = range(x)
#     print_final_performance(df, x_values, f'eval_Freeway/{x}_final_perf_results')

# df = pd.read_csv('eval_SpaceInvaders/eval_results.csv')
# for x in range(1,11):
#     x_values = range(x)
#     print_final_performance(df, x_values, f'eval_SpaceInvaders/{x}_final_perf_results')

df = pd.read_csv('eval_MetaWorld/eval_results.csv')
print(df.keys())
for x in range(1,21):
    x_values = range(x)
    print_ratio_MetaWorld(df, x_values, f'eval_MetaWorld/{x}_forgetting_results')

Index(['algorithm', 'test task', 'train task', 'seed', 'success'], dtype='object')
       algorithm    0  mean
0         cbpnet  0.0   0.0
1          creus  0.0   0.0
2       finetune  0.0   0.0
3        fusenet  0.0   0.0
4  fusenet_merge  0.0   0.0
5        masknet  0.0   0.0
6        packnet  0.0   0.0
7        prognet  0.0   0.0
8         simple  0.0   0.0
       algorithm    0    1  mean
0         cbpnet  1.0  0.0  0.50
1          creus  1.0  0.0  0.50
2       finetune  0.0  0.0  0.00
3        fusenet  0.0  0.0  0.00
4  fusenet_merge  0.0  0.0  0.00
5        masknet  0.0  0.0  0.00
6        packnet  0.0  0.0  0.00
7        prognet  0.0  0.0  0.00
8         simple -0.3  0.0 -0.15
       algorithm    0    1    2      mean
0         cbpnet  1.0  0.0  0.0  0.333333
1          creus  1.0  0.0  0.0  0.333333
2       finetune  0.0  1.0  0.0  0.333333
3        fusenet -0.2  1.0  0.0  0.266667
4  fusenet_merge  0.0  0.0  0.0  0.000000
5        masknet  0.0  0.0  0.0  0.000000
6        pack

/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: T

       algorithm    0    1    2    3    4  mean
0         cbpnet  1.0  0.0  1.0  0.0  0.0  0.40
1          creus  1.0  0.0  1.0  0.0  0.0  0.40
2       finetune  0.0  1.0  1.0  1.0  0.0  0.60
3        fusenet -0.3  1.0  1.0  0.0  0.0  0.34
4  fusenet_merge  0.0  0.0  1.0  1.0  0.0  0.40
5        masknet  0.0  0.0  1.0  0.0  0.0  0.20
6        packnet  0.0  0.0  0.0  0.0  0.0  0.00
7        prognet  0.0  0.0  1.0  1.0  0.0  0.40
8         simple  0.0  0.0  1.0  1.0  0.0  0.40
       algorithm    0    1    2    3    4    5      mean
0         cbpnet  1.0  0.0  1.0  0.0  0.0  0.0  0.333333
1          creus  1.0  0.0  1.0  0.0  0.0  0.0  0.333333
2       finetune  0.0  1.0  1.0  1.0  0.0  0.0  0.500000
3        fusenet  0.0  1.0  1.0  0.0  0.0  0.0  0.333333
4  fusenet_merge -0.1  0.0  1.0  1.0  0.0  0.0  0.316667
5        masknet  0.0  0.0  1.0  0.0  0.0  0.0  0.166667
6        packnet  0.0  0.0  0.0  0.0  0.0  0.0  0.000000
7        prognet  0.0  0.0  1.0  1.0  0.0  0.0  0.333333
8      

/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: T

       algorithm    0    1      2    3    4    5    6    7    8      mean
0         cbpnet  1.0  0.0  0.625  0.0  0.0  1.0  0.0  0.0  0.0  0.291667
1          creus  1.0  0.0  1.000  0.0  0.0  1.0  0.0  0.0  0.0  0.333333
2       finetune  0.0  1.0  1.000  1.0  0.0  1.0  0.0  0.0  0.0  0.444444
3        fusenet  0.0  1.0  1.000  0.0  0.0  0.7  0.0  0.0  0.0  0.300000
4  fusenet_merge  0.0  0.0  1.000  1.0  0.0  1.0  0.0  0.0  0.0  0.333333
5        masknet  0.0  0.0  1.000  0.0  0.0  1.0  0.0  0.0  0.0  0.222222
6        packnet  0.0  0.0  0.000  0.0  0.0  1.0  0.0  0.0  0.0  0.111111
7        prognet  0.0  0.0  1.000  1.0  0.0  1.0  1.0  0.0  0.0  0.444444
8         simple  0.0  0.0  0.500  1.0  0.0  1.0  0.0  0.0  0.0  0.277778
       algorithm    0    1         2    3    4    5    6    7    8    9  \
0         cbpnet  1.0  0.0  1.000000  0.0  0.0  1.0  0.0  0.0  0.7  0.0   
1          creus  1.0  0.0  1.000000  0.0  0.0  1.0  0.0  0.0  1.0  0.0   
2       finetune  0.0  1.0  1.00000

...

/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: T

       algorithm    0    1    2    3    4    5    6    7    8    9   10   11  \
0         cbpnet  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0   
1          creus -3.5  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0 -3.5  0.0   
2       finetune  0.0  1.0  1.0  1.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0   
3        fusenet  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0   
4  fusenet_merge  0.0  0.0  1.0  1.0  0.0  0.9  0.0  0.0  1.0  1.0  1.0  1.0   
5        masknet -0.8  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0   
6        packnet  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0   
7        prognet  0.0  0.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0  1.0  0.0   
8         simple  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

    12   13   14   15   16      mean  
0  1.0  1.0  0.0  1.0  0.0  0.529412  
1  1.0  1.0  0.0  1.0  0.0 -0.058824  
2  1.0  0.0  0.0  1.0  0.0  0.470588  
3  1.0  0.0  0.0  1.0  0.0  0.470588  
4  1

/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: T

       algorithm    0    1    2    3    4         5    6    7         8  ...  \
0         cbpnet  1.0  0.0  1.0  0.0  0.0  1.000000  0.0  0.0  0.000000  ...   
1          creus  1.0  0.0  1.0  0.0  0.0  0.888889  0.0  0.0  0.000000  ...   
2       finetune  0.0  1.0  1.0  1.0  0.0  1.000000  0.0  0.0 -0.428571  ...   
3        fusenet  0.0  1.0  1.0  0.0  0.0  1.000000  0.0  0.0  0.000000  ...   
4  fusenet_merge  0.0  0.0  1.0  1.0  0.0  1.000000  0.0  0.0 -0.666667  ...   
5        masknet  0.0  0.0  1.0  0.0  0.0  0.700000  0.0  0.0  1.000000  ...   
6        packnet  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.666667  ...   
7        prognet  0.0  0.0  1.0  1.0  0.0  1.000000  1.0  0.0  0.000000  ...   
8         simple  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.000000  ...   

         10   11   12   13   14        15   16   17   18      mean  
0  1.000000  0.0  1.0  1.0  0.0  1.000000  0.0  0.0  0.0  0.421053  
1  1.000000  0.0  1.0  1.0  0.0  0.900000  0.0  0.0  0.0  0.3

/tmp/ipykernel_56615/4124213180.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
